Preparing to submit PacBio long read data for late 2018 early 2019

Link to sheet

https://docs.google.com/spreadsheets/d/1GBMmL3Mmzwe_chp-jOxEfKo7jK70B90rpA3nOz-qPyU/edit?ts=5ea1fd7b#gid=1718836302

In [ ]:
import os
import sys
import requests
import pandas
import paramiko
import json
from IPython import display
from pathlib import Path

In [ ]:
from curation_common import *
from htsworkflow.submission.encoded import DCCValidator

In [ ]:
GCATDIR = os.path.expanduser('~/src/gcat')
if GCATDIR not in sys.path:
    sys.path.append(GCATDIR)
    import gcat

In [ ]:
from htsworkflow.submission.encoded import Document
from htsworkflow.submission.aws_submission import run_aws_cp

In [ ]:
# live server & control file
server = ENCODED('www.encodeproject.org')
spreadsheet_name = Path('~/woldlab/ENCODE/CastHybridES.ods').expanduser()

# test server & datafile
#server = ENCODED('test.encodedcc.org')
#spreadsheet_name = os.path.expanduser('~diane/woldlab/ENCODE/C1-encode3-limb-2017-testserver.ods')

server.load_netrc()
validator = DCCValidator(server)

In [ ]:
award = 'UM1HG009443'

# Register Biosamples

In [ ]:
biosample = pandas.read_excel(spreadsheet_name, sheet_name='Biosamples', header=0, engine='odf')
created = server.post_sheet('/biosamples/', biosample, 
                            verbose=True, 
                            dry_run=True, 
                            validator=validator)
print(len(created))

In [ ]:
if created:
    biosample.to_excel('/dev/shm/biosamples.xlsx', index=False)

# Register Libraries

In [ ]:
print(spreadsheet_name)
book = gcat.get_file(spreadsheet_name, fmt='pandas_excel')
libraries = pandas.read_excel(spreadsheet_name, sheet_name='Libraries', header=0, engine='ods')
libraries = book.parse('Libraries', header=0)
created = server.post_sheet('/libraries/', 
                            libraries, 
                            verbose=True, 
                            dry_run=True, 
                            validator=validator)
print(len(created))

In [ ]:
if created:
    libraries.to_excel('/dev/shm/libraries.xlsx', index=False)

# Register Experiments

In [ ]:
print(server.server)
book = gcat.get_file(spreadsheet_name, fmt='pandas_excel')
experiments = book.parse('Experiments', header=0)
experiments = experiments[experiments['accession'] != 'barbara approval needed']
created = server.post_sheet('/experiments/', 
                            experiments, 
                            verbose=True, 
                            dry_run=True, 
                            validator=validator)
print(len(created))

In [ ]:
if created:
    experiments.to_excel('/dev/shm/experiments.xlsx', index=False)

# Register Replicates

In [ ]:
print(server.server)
print(spreadsheet_name)
book = gcat.get_file(spreadsheet_name, fmt='pandas_excel')
replicates = book.parse('Replicates', header=0)
replicates = replicates[replicates['uuid'] != 'barbara approval needed']
created = server.post_sheet('/replicates/',
                            replicates, 
                            verbose=True, 
                            dry_run=True, 
                            validator=validator)
print(len(created))

In [ ]:
if created:
    replicates.to_excel('/dev/shm/replicates.xlsx', index=False)

In [ ]:
with pandas.ExcelWriter(f'/dev/shm/submission.xlsx') as book:
    biosample.to_excel(book, sheet_name='Biosample', index=False)
    experiments.to_excel(book, sheet_name='Experiment', index=False)
    libraries.to_excel(book, sheet_name='Library', index=False)
    replicates.to_excel(book, sheet_name='Replicate', index=False)
    #file_sheet.to_excel(book, sheet_name='File', index=False)
 

# What are the biosamples?

Are the libraries linked reasonably?

In [ ]:
pb82 = server.get_json('ali-mortazavi:PB82')
print(pb82['@id'], pb82['aliases'])
print(pb82['biosample']['@id'], pb82['biosample']['summary'])
pb82_rep = server.get_json(pb82['replicates'][0])
print(pb82_rep['biological_replicate_number'], pb82_rep['technical_replicate_number'])

In [ ]:
pb83 = server.get_json('ali-mortazavi:PB83')
print(pb83['@id'], pb83['aliases'])
print(pb83['biosample']['@id'], pb83['biosample']['summary'])
pb83_rep = server.get_json(pb83['replicates'][0])
print(pb83_rep['biological_replicate_number'], pb83_rep['technical_replicate_number'])

In [ ]:
pb84 = server.get_json('ali-mortazavi:PB84')
print(pb84['@id'], pb84['aliases'])
print(pb84['biosample']['@id'], pb84['biosample']['summary'])
pb84_rep = server.get_json(pb84['replicates'][0])
print(pb84_rep['biological_replicate_number'], pb84_rep['technical_replicate_number'])

In [ ]:
pb85 = server.get_json('ali-mortazavi:PB85')
print(pb85['@id'], pb85['aliases'])
print(pb85['biosample']['@id'], pb85['biosample']['summary'])
pb85_rep = server.get_json(pb85['replicates'][0])
print(pb85_rep['biological_replicate_number'], pb85_rep['technical_replicate_number'])